## Chapter 6 : Hands-On usage of b2in eventing, BRIL common dataformats

For inter-process communication, we use b2in eventing in BRIL. b2in is a TCP-based data transport protocol. Eventing is a word to describe the publisher/subscriber model. The publishing and subscribing processes are decoupled, they do not know each other. They exchange messages via eventing servers which can be viewed abstractly as if there is a common data "bus" between data providers and consumers. The category of all messages of the same type is called a "topic". Multiple publisher can publish to the same topic and multiple subscribers can subscribe to it.

One b2in message consists two parts: binary data and string map metadata. Data part presents as References\*, and metadata xdata::Properties. Unpon receving a b2in message, one should parse the binary data from the message handle References\*, and use the metadata in xdata::Properties if needed. To publish a message, one must construct a binary data buffer with the correct values and put it in References\* then hand it the messenger together with necessary metadata. It is very **important** to understand the ownership of the messages on receiving and publishing.

On receiving a message, your message callback function **acquires** the ownership of the message handle, it must release the message memory after done with it. We recommend using a AutoReference guard to help achieving this.

On calling message publish, your function has **handed** over the ownership of the message to the messenger. You should NOT release it again. However, in the case that the publish failed with exception, you should release the message by yourself as part of the exception handling. 

(Above statements may change with future xdaq releases.)  

BRIL daq defines a common data format to facilitate binary message exchange. They are defined in **daq/interface/bril/include**. There are two types of BRIL topics, simple or compound. They have a common header and a portion of binary payload. The payload carried by the simple topic is a c-array of primitive types. The payload carried by the compound topic contains mixed type. The memory layout of the compound type payload is defined by a data dictionary string. And this dictionary string must be sent together as the message metadata. Given the data dictionary, a compound data streamer (provided) can decode the compound data accordingly. 

For the metadata part of the message, the following are the agreed property for BRIL topics:

```
NOSTORE "1" or empty , a tag for the storage manage receving the message to ignore or store the message

PAYLOAD_DICT "payload dictionary"

DATA_VERSION "BRIL data definition version"
```
### Example 


Export again from svn example directory and set up your working environment as described in Chapter 2.

Choose either modify the configuration file by hand $workdir/daq/bril/mypackage/xml/DataAnalyzer.xml 

*or* 

generate it from template:

```
cd $workdir/daq/bril/scripts

./generatexml.sh mypackage/DataAnalyzer

```

Build mypackage library:


```
make install

```

Go to where the configuration xml resides and launch the executive from there:

```
/opt/xdaq/bin/xdaq.exe -p $your_context_port -e /opt/xdaq/etc/default.profile -c `pwd`/DataAnalyzer.xml
```

Observe the different publication frequency.

The xml configuration file contains more **application blocks** and more **ports** than before, what are they?

How do I guarantee the received messages are released after usage? What happens if I do not release them or hold them for a long time?

What happens if I configured the publisher wrong so that my publish call fails over and over again?

Should I care about the thread-safety of my data cach in this particular setup?

### Excercise

We inherit from different base classes in this example, what they are and what the purpose of each?

Write a simple receiver to demonstrate that the message you published are correct.

Comment out the ref guard in the onMessage call and monitor its memory footprint, using top command or the memprobe service in the BRIl zone.